# Data wrangling

This notebook loads the different datasets used in the analysis into a single NETCDF4 file, with descriptive attributes maintained for each dataset. The datasets used in this notebook are listed below. The output file is accessible in the google bucket for this jupyter book and loaded in each notebook. 
 
**Input**:
 - [ICESat-2 monthly gridded sea ice data](https://icesat-2.gsfc.nasa.gov/sea-ice-thickness-data)
 - [Monthly NSIDC sea ice concentration data](https://nsidc.org/data/g02202)
 - [NSIDC region mask](https://nsidc.org/data/polar-stereo/tools_masks.html#region_masks) and [coordinate](https://nsidc.org/data/polar-stereo/tools_geo_pixel.html) tools (psn25 v3)
 - [ERA 5 climate renanalysis data](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels-monthly-means?tab=overview)
 - [PIOMAS mean monthly ice thickness](http://psc.apl.uw.edu/research/projects/arctic-sea-ice-volume-anomaly/)
 - [NSIDC sea ice motion vectors](https://nsidc.org/data/nsidc-0116)
     

**Output**: 
 - NETCDF4 file 

# Interpolate missing ICESat-2 data 
Interpolate missing ICESat-2 data using the [scipy.griddata.interpolate](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.griddata.html) function and add as data variables to the dataset. Because ICESat-2 doesn't provide full monthly coverage, interpolating fills missing grid cells with a best guess based on surrounding data. This helps avoid sampling biases when performing time series analyses, with the cavaet that this interpolation method is subjective. 

In [ ]:
#list of variables to interpolate
IS2VarList = ['ice_type','ice_thickness','snow_depth','freeboard','ice_thickness_unc','snow_density','ice_density']
for varStr in IS2VarList:
    varFilled = []
    for month in range(len(is2.time)): 
        monthlyVar = is2[varStr].values[month]
        if varStr == 'ice_thickness':   #if var is ice_thickness_int, set ice_thickness to zero if ice_thickness is NaN and sea ice concentration < 15%
            monthlyVar[seaice_conc_monthly_cdr.values[month] <= 0.15] = 0
            
        #don't interpolate over land (regionMask = 20) or the Canadian Archipelago (regionMask = 14)
        monthlyVar = ma.masked_where((np.isnan(monthlyVar)) & (regionMask != 20) & (regionMask != 14) & (seaice_conc_monthly_cdr.values[month] > 0.15), monthlyVar)
        interp = scipy.interpolate.griddata((is2Lons[~monthlyVar.mask], is2Lats[~monthlyVar.mask]), 
                    monthlyVar[~monthlyVar.mask].flatten(),(is2Lons, is2Lats), method = 'nearest')
        varFilled.append(interp)
    
    #convert varFilled to a DataArray object 
    varFilledDataArray = xr.DataArray(data = varFilled, 
                                      dims = ['time', 'y', 'x'], 
                                      attrs = is2[varStr].attrs)
    varFilledDataArray.attrs['note'] = 'interpolated using nearest neighbor method'
    is2[varStr + '_interp'] = varFilledDataArray

#Add descriptive attributes & coordinates to all ICESat-2 variables    
for var in is2.data_vars:
    is2[var] = is2[var].assign_attrs(is2.attrs)

#drop lat and lon as data variables and add as coordinate values 
is2 = is2.assign_coords(coords = {'latitude': (('y','x'), is2Lats), 'longitude': (('y','x'), is2Lons)})

```{note}
This notebook is **NOT** configured to run in Google Colab. Each individual dataset used to compile the final data product can be downloaded from the links above. The files generated by this notebook is provided in the google bucket for this book. [Click here](https://storage.googleapis.com/icesat2-book-data/icesat2-book-dataset.nc) to download or view data. 
```

# Import notebook dependencies

In [1]:
import os
import numpy as np
import numpy.ma as ma
import xarray as xr
import pandas as pd
import scipy.interpolate
import pyproj
from datetime import date
from glob import glob
import sys

# Ignore warnings in the notebook to improve display
import warnings
warnings.filterwarnings('ignore')

# Read in data

## Define filepaths
Define filepaths to data on your local machine

In [2]:
IS2_path = ''         #path to monthly gridded ICESat-2 data
SIC_path = ''         #path to NSIDC weekly sea ice concentration data
regionMask_path = ''  #path to NSIDC region mask for the Arctic
ERA5_path = ''        #path to ERA5 climate reanalysis data 
PIOMAS_path = ''      #path to PIOMAS data
drift_path = ''       #path to NSIDC sea ice drift data 

Filepaths on Nicole's computer; cells removed in jupyter book

In [3]:
#path to directory of data
localDirectory = '/Users/nicolekeeney/github_repos/icesat2-book-data/'

#IS2_path = localDirectory + 'cpom_thickness/'             #Alek's v1 data
#IS2_path = localDirectory + 'v2icesat2thickness_nov2020/' #Alek's v2 data 
IS2_path = localDirectory + 'IS2SITMOGR4/'                 #Alek's v3 data for monthly gridded ICESat-2 data
SIC_path = localDirectory + 'cdr/'                         #path to NSIDC weekly sea ice concentration data
regionMask_path = localDirectory + 'regionMask/'           #path to NSIDC region mask for the Arctic
ERA5_path = localDirectory  + 'ERA5/ERA5-temp-and-rad.nc'  #path to ERA5 climate reanalysis data 
PIOMAS_path = localDirectory + 'PIOMAS_monthly_thickness/' #path to PIOMAS data
drift_path = localDirectory + 'drifts/'                    #path to NSIDC sea ice drift data 

## Select for only winter months
Since we are interested in winter sea ice growth, we restrict our datasets to the months of November, December, January, February, March, and April. The function below returns a list of pandas date ranges for Nov-Apr between start and end dates of interest. 

In [4]:
def getWinterDateRange(start_year, end_year): 
    """ Gets date range for winter season/s
    Args: 
        start_year (int): start year 
        end_year (int): end year 
        
    Returns: 
        winters (list): list of dates for all winter seasons in the input range (i.e: ['1980-11','1980-12','1981-01',
         '1981-02','1981-03','1981-04')
    """
    winters = []
    for year in range(start_year, end_year, 1):
        winters += pd.date_range(start = str(year) + '-11', end = str(year + 1) + '-04', freq = 'MS')
    return winters

In [5]:
startYear = 2018
endYear = 2020
winters = getWinterDateRange(startYear, endYear) #get date range for winter 18-19 and winter 19-20
print(winters)

[Timestamp('2018-11-01 00:00:00', freq='MS'), Timestamp('2018-12-01 00:00:00', freq='MS'), Timestamp('2019-01-01 00:00:00', freq='MS'), Timestamp('2019-02-01 00:00:00', freq='MS'), Timestamp('2019-03-01 00:00:00', freq='MS'), Timestamp('2019-04-01 00:00:00', freq='MS'), Timestamp('2019-11-01 00:00:00', freq='MS'), Timestamp('2019-12-01 00:00:00', freq='MS'), Timestamp('2020-01-01 00:00:00', freq='MS'), Timestamp('2020-02-01 00:00:00', freq='MS'), Timestamp('2020-03-01 00:00:00', freq='MS'), Timestamp('2020-04-01 00:00:00', freq='MS')]


## ICESat-2 monthly gridded sea ice data

### Read data

In [6]:
def getIS2Data(dataPath, dates): 
    """Gets ICESat-2 data for provided date range
    
    Args: 
        dataPath (str): path to local directory of ICESat-2 data 
        dates (list): pandas Timestamp objects generated by getWinterDateRange
        
    Returns: 
        is2 (xarray dataset): ICESat-2 data or NONE if file does not exist for inputted date range
    """
    is2List = [] #empty list for compiling xarray DataArray objects
    for date in dates: 
        try:
            filename = glob(dataPath+ 'IS2*' + date.strftime('%y') + date.strftime('%m') + '*.nc')[0]
        except: 
            print('Cannot find files; check date range, filepath or if glob is imported')
            return None
        is2 = xr.open_dataset(filename)
        is2 = is2.assign_coords({'time': date})
        is2List.append(is2)
    is2Data = xr.concat(is2List, dim = 'time') #concatenate all DataArray objects into a single DataArray
    
    return is2Data

In [11]:
is2 = getIS2Data(IS2_path, winters)
is2 = getIS2Data(localDirectory + "/old_is2/v2icesat2thickness_nov2020/", winters)
is2

<xarray.Dataset>
Dimensions:            (time: 12, x: 448, y: 304)
Coordinates:
  * time               (time) datetime64[ns] 2018-11-01 ... 2020-04-01
Dimensions without coordinates: x, y
Data variables: (12/14)
    projection         (time) int32 -2147483647 -2147483647 ... -2147483647
    longitude          (time, x, y) float32 168.3 168.1 168.0 ... -10.18 -9.999
    latitude           (time, x, y) float32 31.1 31.2 31.3 ... 34.68 34.58 34.47
    xgrid              (time, x, y) float32 -3.838e+06 -3.812e+06 ... 3.738e+06
    ygrid              (time, x, y) float32 5.838e+06 5.838e+06 ... -5.338e+06
    ice_thickness      (time, x, y) float32 nan nan nan nan ... nan nan nan nan
    ...                 ...
    mean_day_of_month  (time, x, y) float32 nan nan nan nan ... nan nan nan nan
    snow_depth         (time, x, y) float32 nan nan nan nan ... nan nan nan nan
    snow_density       (time, x, y) float32 nan nan nan nan ... nan nan nan nan
    freeboard          (time, x, y) float32 nan nan nan nan ... nan nan nan nan
    ice_type           (time, x, y) float32 nan nan nan nan ... nan nan nan nan
    ice_density        (time, x, y) float32 nan nan nan nan ... nan nan nan nan
Attributes:
    contact:      Alek Petty (alek.a.petty@nasa.gov)
    description:  Gridded ICESat-2  Nov 2018 sea ice thickness (and ancillary...
    reference:    Petty, A. A., N. T. Kurtz, R. Kwok, T. Markus, T. A. Neuman...
    history:      Created 18/11/20

In [8]:
is2

<xarray.Dataset>
Dimensions:            (time: 12, x: 304, y: 448)
Coordinates:
  * time               (time) datetime64[ns] 2018-11-01 ... 2020-04-01
Dimensions without coordinates: x, y
Data variables: (12/14)
    projection         (time) int32 -2147483647 -2147483647 ... -2147483647
    longitude          (time, y, x) float32 168.3 168.1 168.0 ... -10.18 -9.999
    latitude           (time, y, x) float32 31.1 31.2 31.3 ... 34.68 34.58 34.47
    xgrid              (time, y, x) float32 -3.838e+06 -3.812e+06 ... 3.738e+06
    ygrid              (time, y, x) float32 5.838e+06 5.838e+06 ... -5.338e+06
    ice_thickness      (time, y, x) float32 nan nan nan nan ... nan nan nan nan
    ...                 ...
    mean_day_of_month  (time, y, x) float32 nan nan nan nan ... nan nan nan nan
    snow_depth         (time, y, x) float32 nan nan nan nan ... nan nan nan nan
    snow_density       (time, y, x) float32 nan nan nan nan ... nan nan nan nan
    freeboard          (time, y, x) float32 nan nan nan nan ... nan nan nan nan
    ice_type           (time, y, x) float32 nan nan nan nan ... nan nan nan nan
    ice_density        (time, y, x) float32 nan nan nan nan ... nan nan nan nan
Attributes:
    contact:      Alek Petty (alek.a.petty@nasa.gov)
    description:  Gridded Nov 2018 Arctic sea ice thickness and ancillary dat...
    reference:    Data doi: 10.5067/CV6JEXEE31HF. Original methodology descri...
    history:      Created 22/04/21

### Clean data and extract useful variables

In [8]:
#drop projection variable 
is2 = is2.drop('projection')

#get lat and lon
is2Lats = is2.latitude.isel(time = 0).values
is2Lons = is2.longitude.isel(time = 0).values
is2LonsAttrs = is2.longitude.attrs 
is2LatsAttrs = is2.latitude.attrs

#assign lat and lon as coordinates to dataset
is2 = is2.assign_coords(coords = {'latitude': (('y','x'), is2Lats), 'longitude': (('y','x'), is2Lons)})

## NSIDC sea ice concentration data 

### Read data 

In [45]:
def getSICData(dataPath, dates): 
    """Gets sea ice concentration data for provided date range
    
    Args: 
        dataPath (str): path to local directory of sea ice concentration data 
        dates (list): pandas Timestamp objects generated by getWinterDateRange
        
    Returns: 
        sicData (xarray dataset): sea ice concentration data or NONE if file does not exist for inputted date range
    """
    sicList = [] #empty list for compiling xarray DataArray objects
    for date in dates: 
        try:
            filename = glob(dataPath+ 'seaice_conc_monthly*' + date.strftime('%Y') + date.strftime('%m') + '*.nc')[0]
        except: 
            print('Cannot find files; check date range, filepath or if glob is imported')
            return None
        dropVariables = ['goddard_merged_seaice_conc_monthly','goddard_bt_seaice_conc_monthly', 'goddard_nt_seaice_conc_monthly']
        sicList.append(xr.open_dataset(filename, drop_variables = dropVariables))
    sicData = xr.concat(sicList , dim = 'time', combine_attrs = 'override', data_vars = 'minimal') #concatenate all DataArray objects into a single DataArray
    
    return sicData

In [46]:
sic = getSICData(SIC_path, winters)

OSError: [Errno -101] NetCDF: HDF error: b'/Users/nicolekeeney/github_repos/icesat2-book-data/cdr/seaice_conc_monthly_nh_f17_201811_v03r01.nc'

### Clean data
Clean data by removing flagged data and filling the pole hole. We assume concentration is 100% within the pole hole because we are only looking at winter data. 

In [11]:
#remove flagged data 
sic_monthly_cdr = sic['seaice_conc_monthly_cdr'].where(sic['seaice_conc_monthly_cdr'] < 0)

#fill pole hole as 100% concentration
sic_monthly_cdr = sic['seaice_conc_monthly_cdr'].where(sic['latitude'] < 88, 1)

#reassign variable
sic = sic.assign(seaice_conc_monthly_cdr = sic_monthly_cdr)

#reassign dimensions 
seaice_conc_monthly_cdr = xr.DataArray(data = sic['seaice_conc_monthly_cdr'], dims = ['time', 'y', 'x'], coords = {'time': winters, 'latitude': (('y','x'), is2Lats), 'longitude': (('y','x'), is2Lons)}) 

#attributes from entire NSIDC data to maintain
desiredSICAttrs = ['title', 'references', 'contributor_name', 'license', 'summary'] #attributes to maintain from entire sea ice concentration dataset
SICDatasetAttrs = {x:sic.attrs[x] for x in desiredSICAttrs}
seaice_conc_monthly_cdr = seaice_conc_monthly_cdr.assign_attrs(SICDatasetAttrs)

#add to is2 dataset 
is2['seaice_conc_monthly_cdr'] = seaice_conc_monthly_cdr

## NSIDC region mask for the Arctic data

### Read data

In [12]:
def getRegionMask(dataPath): 
    """Gets NSIDC region mask for map projection
    
     Args:
         dataPath (str): path to NSIDC region mask
         
     Returns: 
         shapedMask (numpy array): NSIDC arctic region mask gridded to shape [448, 304]
         shapedLons (numpy array): longitudes gridded to shape [448, 304]
         shapedLats (numpy array): latitudes gridded to shape [448, 304] 
    """ 
    gridShape = [448, 304] #shape of grid to reshape data to 
    
    regionMask = open(dataPath + '/sect_fixed_n.msk', 'rb') #open region mask 
    shapedMask = np.reshape(np.fromfile(file = regionMask, dtype='uint8'), gridShape) #reshape mask to grid shape
    
    maskLons = open(dataPath + '/psn25lons_v3.dat', 'rb') #open region mask longitudes
    maskLats = open(dataPath + '/psn25lats_v3.dat', 'rb') #open region mask latitudes
    shapedLons = np.reshape(np.fromfile(file = maskLons, dtype='<i4')/100000., gridShape) #reshape longitudes to grid shape
    shapedLats = np.reshape(np.fromfile(file = maskLats, dtype='<i4')/100000., gridShape) #reshape latitudes to grid shape

    return shapedMask, shapedLons, shapedLats

In [13]:
regionMask, maskLons, maskLats = getRegionMask(regionMask_path)

### Define descriptive information
These variables will be used later for creating the dataset. 

In [14]:
#coords and attributes for Region Mask
regionMaskCoords = {'region_mask': (('y','x'), regionMask)}
regionMaskKeys = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 20, 21])
regionMaskLabels = np.array(['non-region oceans', 'Sea of Okhotsk and Japan','Bering Sea','Hudson Bay','Gulf of St. Lawrence',
                    'Baffin Bay, Davis Strait & Labrador Sea','Greenland Sea', 'Barents Seas','Kara Sea','Laptev Sea','East Siberian Sea',
                    'Chukchi Sea','Beaufort Sea','Canadian Archipelago','Arctic Ocean','Land','Coast'])
regionMaskAttrs = {'description': 'NSIDC region mask for the Arctic', 'keys': regionMaskKeys, 'labels' : regionMaskLabels, 'note': 'keys and labels ordered to match by index'}

## ERA5 climate reanalysis data 

### Read data

In [15]:
ERA5 = xr.open_dataset(ERA5_path)

### Clean data
Drop unneeded variables and converting temperature to Celcius

In [16]:
#remove unneeded expver variable. 
#for more info on the exper variable, see https://confluence.ecmwf.int/pages/viewpage.action?pageId=173385064
ERA5 = ERA5.sel(expver = 1)
ERA5 = ERA5.drop('expver')

#select data from past two winters 
ERA5 = ERA5.sel(time = getWinterDateRange(2018, 2020))
ERA5 = ERA5.assign_coords(time = getWinterDateRange(2018, 2020))

#convert t2m temp to celcius 
tempCelcius = ERA5['t2m'] - 283.15
tempCelcius.attrs['units'] = 'C' #change units attribute to C (Celcius)
tempCelcius.attrs['long_name'] = '2 meter temperature'
ERA5 = ERA5.assign(t2m = tempCelcius) #add to dataset as a new data variable

#add descriptive attributes 
ERA5.attrs = {'description': 'era5 monthly averaged data on single levels from 1979 to present', 
              'website': 'https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels-monthly-means?tab=overview', 
              'contact': 'copernicus-support@ecmwf.int',
             'citation': 'Copernicus Climate Change Service (C3S) (2017): ERA5: Fifth generation of ECMWF atmospheric reanalyses of the global climate . Copernicus Climate Change Service Climate Data Store (CDS), July 2020. https://cds.climate.copernicus.eu/cdsapp#!/home'}

#restrict ERA5 data to the Arctic 
ERA5 = ERA5.where(ERA5.latitude > 50)

## PIOMAS sea ice thickness data 

### Read data

In [51]:
def getPIOMASData(dataPath, startYear = 1978, endYear = 2020): 
    """Gets PIOMAS mean monthly sea ice thickness data for an input time range. 
    
    Args: 
        dataPath (str): path to local directory of PIOMAS data 
        startYear (int): year to start loading data from (defaul to 1978)
        endYear (int): year to stop loading data from (default to 2020)
    
    Returns: 
        PIOMAS_data (xarray DataArray): PIOMAS data with descriptive coordinates and attributes
        
    Note: last available PIOMAS data was from July 2020 at the time of creation of this function
    """
    dataList = [] #empty list for compiling data

    for year in range(startYear, endYear + 1, 1):     
        data = open(dataPath + 'heff.H' + str(year), 'rb') 
        if year == 2020: #need special reshaping for 2020 because we dont have a full year of data
            period = 7
        else:
            period = 12
        dataList += list(np.fromfile(file = data, dtype='f').reshape([period, 120, 360]))
    
    #add latitude and longitude 
    gridP = np.loadtxt(dataPath + 'grid.dat.txt')
    lonsP = gridP[0:4320, :].flatten()
    lonsP = np.reshape(lonsP, [120,360])
    latsP = gridP[4320:, :].flatten()
    latsP = np.reshape(latsP, [120,360])
    
    #load dataList as an xarray DataArray with descriptive attributes and coordinates
    time = pd.date_range(start = str(startYear), end = str(endYear) + '-07', freq = 'MS')
    PIOMAS_attrs = {'units': 'meters', 'long_name': 'PIOMAS sea ice thickness', 'data description': 'PIOMAS monthly mean sea ice thickness', 'citation': 'Zhang, J.L. and D.A. Rothrock, “Modeling global sea ice with a thickness and enthalpy distribution model in generalized curvilinear coordinates“, Mon. Weather Rev., 131, 845-861, 2003'}
    PIOMAS_data = xr.DataArray(dataList, dims = ['time','y','x'], coords = {'time': time, 'longitude': (('y','x'), lonsP), 'latitude': (('y','x'), latsP)}, attrs = PIOMAS_attrs)
    
    return PIOMAS_data

In [52]:
piomasData = getPIOMASData(PIOMAS_path, startYear = 2018, endYear = 2020)

In [53]:
piomasData

<xarray.DataArray (time: 31, y: 120, x: 360)>
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
...
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 2018-01-01 2018-02-01 ... 2020-07-01
    longitude  (y, x) float64 319.5 320.5 321.5 322.5 ... 317.9 318.4 318.8
    latitude   (y, x) float64 48.99 48.99 48.99 48.99 ... 72.71 72.71 72.71
Dimensions without coordinates: y, x
Attributes:
    units:             meters
    long_name:         PIOMAS sea ice thickness
    data description:  PIOMAS monthly mean sea ice thickness
    citation:          Zhang, J.L. and D.A. Rothrock, “Modeling global sea ic...

## NSIDC sea ice drift data 

### Read data

In [19]:
def getDriftData(dataPath):
    """Gets weekly NSIDC sea ice drift data for the Arctic, resamples monthly, and returns a dataset
    
    Args: 
        dataPath (str): path to local directory of weekly drift data 

    Returns: 
        monthlyDrifts (xarray DataArray): sea ice drift data, resampled to monthly means, with descriptive coordinates and attributes
        
    Note: This function uses two different datasets from the NSIDC for drift in order to get the most recent data. Drift dataset from 1978-2018 was combined with quicklook drift dataset from 2019-present, maintaining attitributes of the 1978-2018 dataset
    """
    def get_uv_from_xy(xdrift, ydrift, lon):
        """convert the drift vectors to zonal/meridional, and calculate magnitude
        """
        alpha = lon*np.pi/180. #convert longitudes to radians 
        uvelT = ydrift*np.sin(alpha) + xdrift*np.cos(alpha)
        vvelT = ydrift*np.cos(alpha) - xdrift*np.sin(alpha)
        vectorMag = np.sqrt(uvelT**2 + vvelT**2)
        return uvelT, vvelT, vectorMag
    
    #combine nc weekly files into a single xarray dataset
    files = [xr.open_dataset(dataPath + f) for f in os.listdir(dataPath) if os.path.isfile(dataPath + f) and f.endswith('.nc')]
    weeklyDrifts = xr.concat(files, dim = 'time')

    #get transformed u,v variables and add to dataset
    uvelT, vvelT, vectorMag = get_uv_from_xy(weeklyDrifts.u, weeklyDrifts.v, weeklyDrifts.longitude)
    weeklyDrifts = weeklyDrifts.assign(drifts_uT = uvelT, drifts_vT = vvelT, drifts_magnitude = vectorMag)
    
    #resample to get monthly data 
    monthlyDrifts = weeklyDrifts.resample(time='MS', keep_attrs = True).mean()

    #convert to same time format as ICESat-2 
    monthlyDrifts = monthlyDrifts.assign_coords(time = [pd.to_datetime(date.strftime('%m-%d-%Y')) for date in monthlyDrifts.time.values])
    
    #add attributes
    monthlyDrifts.drifts_uT.attrs = {'description':'along-x component of the ice motion (u variable) converted to zonal/meridional', 'units':'cm/s', 'long_name':'sea ice x velocity'}
    monthlyDrifts.drifts_vT.attrs = {'description':'along-y component of the ice motion (v variable) converted to zonal/meridional', 'units':'cm/s', 'long_name':'sea ice y velocity'}
    monthlyDrifts.drifts_magnitude.attrs = {'long_name': 'drift vector magnitude', 'units':'cm/s'}
    monthlyDrifts.attrs['njkeeney comment'] = 'drift dataset from 1978-2018 was combined with quicklook drift dataset from 2019-present, maintaining attitributes of the 1978-2018 dataset'
    monthlyDrifts.attrs['citation'] = 'Tschudi, M., W. N. Meier, J. S. Stewart, C. Fowler, and J. Maslanik. 2019. Polar Pathfinder Daily 25 km EASE-Grid Sea Ice Motion Vectors, Version 4. Weekly sea ice motion. Boulder, Colorado USA. NASA National Snow and Ice Data Center Distributed Active Archive Center. doi: https://doi.org/10.5067/INAWUWO7QH7B. July 2020.'
    for var in monthlyDrifts.data_vars: 
        if var not in ['drifts_magnitude','drifts_uT', 'drifts_vT']: 
            monthlyDrifts[var].attrs = weeklyDrifts[var].attrs #resampling function causes attributes to be lost, so add them back here
    
    return monthlyDrifts

In [20]:
drifts = getDriftData(drift_path)   # Read data
drifts = drifts.sel(time = winters) # Restrict data to winter months

# Interpolate missing ICESat-2 data 
Interpolate missing ICESat-2 data using the [scipy.griddata.interpolate](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.griddata.html) function and add as data variables to the dataset. Because ICESat-2 doesn't provide full monthly coverage, interpolating fills missing grid cells with a best guess based on surrounding data. This helps avoid sampling biases when performing time series analyses, with the cavaet that this interpolation method is subjective. 

In [21]:
#list of variables to interpolate
IS2VarList = ['ice_type','ice_thickness','snow_depth','freeboard','ice_thickness_unc','snow_density','ice_density']

#go through variables in list and add as new data variables to ICESat-2 dataset
for varStr in IS2VarList:
    
    #empty list to store monthly interpolated data 
    varFilled = []
    
    for month in range(len(is2.time)): 
        #current month of data 
        monthlyVar = is2[varStr].values[month]
        
        #additional condition for interpolating ice thickness
        if varStr == 'ice_thickness': 
            #if var is ice_thickness_int, set ice_thickness to zero if ice_thickness is NaN and sea ice concentration < 15%
            monthlyVar[seaice_conc_monthly_cdr.values[month] <= 0.15] = 0
        
        #conditions for cells to interpolate
        monthlyVar = ma.masked_where((np.isnan(monthlyVar)) & (regionMask != 20) & (regionMask != 14) & (seaice_conc_monthly_cdr.values[month] > 0.15), monthlyVar)
        
        #append interpolated data to list 
        varFilled.append(scipy.interpolate.griddata((is2Lons[~monthlyVar.mask], is2Lats[~monthlyVar.mask]), 
                    monthlyVar[~monthlyVar.mask].flatten(),(is2Lons, is2Lats), method = 'nearest'))
    
    #convert varFilled to a DataArray object 
    varFilledDataArray = xr.DataArray(data = varFilled, dims = ['time', 'y', 'x'], attrs = is2[varStr].attrs)
    varFilledDataArray.attrs['note'] = 'interpolated from original data'
    
    #add as new data variable to ICESat-2 dataset
    is2[varStr + '_filled'] = varFilledDataArray

#Add descriptive attributes & coordinates to all ICESat-2 variables    
for var in is2.data_vars:
    is2[var] = is2[var].assign_attrs(is2.attrs)

#drop lat and lon as data variables and add as coordinate values 
is2 = is2.assign_coords(coords = {'latitude': (('y','x'), is2Lats), 'longitude': (('y','x'), is2Lons)})

# Regrid all datasets to ICESat-2 grid 
In order to merge ERA5 and PIOMAS data to the same dataset as ICESat-2, it needs to be on the same grid. 

In [22]:
#used in code development, bar hidden in notebook outputs 
def progressBar(i, tot): 
    """Display a progress bar inside a for loop 
    Args: 
        i (int): iteration number
        tot (int): total number of iterations
    """
    j = (i + 1) / tot
    sys.stdout.write('\r [%-20s] %d%% complete' % ('='*int(20*j), 100*j))
    sys.stdout.flush()

In [23]:
def regridToICESat2(dataArrayNEW, xptsNEW, yptsNEW, xptsIS2, yptsIS2):  
    """ Regrid new data to ICESat-2 grid 
    
    Args: 
        dataArrayNEW (xarray DataArray): DataArray to be gridded to ICESat-2 grid 
        xptsNEW (numpy array): x-values of dataArrayNEW projected to ICESat-2 map projection 
        yptsNEW (numpy array): y-values of dataArrayNEW projected to ICESat-2 map projection 
        xptsIS2 (numpy array): ICESat-2 longitude projected to ICESat-2 map projection
        yptsIS2 (numpy array): ICESat-2 latitude projected to ICESat-2 map projection
    
    Returns: 
        gridded (numpy array): data regridded to ICESat-2 map projection
    
    """
    gridded = []
    for i in range(len(dataArrayNEW.values)): 
        monthlyGridded = scipy.interpolate.griddata((xptsNEW.flatten(),yptsNEW.flatten()), dataArrayNEW.values[i].flatten(), (xptsIS2, yptsIS2), method = 'nearest')
        gridded.append(monthlyGridded)
        progressBar(i, len(dataArrayNEW.values))
    return np.array(gridded)

## Regrid ERA5 data
Here, we select a few variables of interest from the ERA5 dataset, regrid them to the ICESat-2 grid, and add them as data variables to the ICESat-2 dataset

In [24]:
# Choose data variables of interest 
ERA5Vars = ['t2m','msdwlwrf']
print('Variables to include: ' + '%s' % ', '.join(map(str, [ERA5[var].attrs['long_name'] for var in ERA5Vars])))

#map projection from ICESat-2 data (can be viewed in is2.projection.attrs['srid'])
IS2_proj = 'EPSG:3411'

#initialize map projection and project data to it
mapProj = pyproj.Proj("+init=" + IS2_proj)
xptsERA, yptsERA = mapProj(*np.meshgrid(ERA5.longitude.values, ERA5.latitude.values))
xptsIS2, yptsIS2 = mapProj(is2Lons, is2Lats)

#grid data
for var in ERA5Vars: 
    #regrid data by calling function
    ERA5gridded = regridToICESat2(ERA5[var], xptsERA, yptsERA, xptsIS2, yptsIS2)
    
    #create xarray DataArray object with descriptive coordinates 
    ERAArray = xr.DataArray(data = ERA5gridded, dims = ['time', 'y', 'x'], coords = {'latitude': (('y','x'), is2Lats), 'longitude': (('y','x'), is2Lons)})
    ERAArray.attrs = ERA5[var].attrs
    ERAArray = ERAArray.assign_attrs(ERA5.attrs)

    #add to ICESat-2 dataset
    is2[var] = ERAArray

Variables to include: 2 meter temperature, Mean surface downward long-wave radiation flux
 [====================] 100% complete

## Regrid PIOMAS data 

In [25]:
#project data to ICESat-2 map projection
xptsPIO, yptsPIO = mapProj(piomasData.longitude.values, piomasData.latitude.values)

#regrid data by calling function
PIOgridded = regridToICESat2(piomasData, xptsPIO, yptsPIO, xptsIS2, yptsIS2)

#create xarray DataArray object with descriptive coordinates 
PIOArray = xr.DataArray(data = PIOgridded, dims = ['time', 'y', 'x'], coords = {'latitude': (('y','x'), is2Lats), 'longitude': (('y','x'), is2Lons)})
PIOArray = PIOArray.assign_coords(time = piomasData.time.values)
PIOArray = PIOArray.assign_attrs(piomasData.attrs)

#restrict data to same time period as ICESat-2
PIOArray_winter = PIOArray.sel(time = winters)

#add to ICESat-2 dataset
is2['PIOMAS_ice_thickness'] = PIOArray_winter

 [====================] 100% complete

## Regrid NSIDC sea ice drift data

In [26]:
#project data to ICESat-2 map projection
xptsDRIFTS, yptsDRIFTS = mapProj(drifts.longitude.values[0], drifts.latitude.values[0])

for var in ['drifts_uT', 'drifts_vT', 'drifts_magnitude']: 
    #regrid data by calling function
    driftsGridded = regridToICESat2(drifts[var], xptsDRIFTS, yptsDRIFTS, xptsIS2, yptsIS2)
    
    #create xarray DataArray object with descriptive coordinates 
    driftsArray = xr.DataArray(data = driftsGridded, dims = ['time', 'y', 'x'], coords = {'latitude': (('y','x'), is2Lats), 'longitude': (('y','x'), is2Lons)})
    driftsArray.attrs = drifts[var].attrs
    driftsArray = driftsArray.assign_attrs(drifts.attrs)

    #add to ICESat-2 dataset
    is2[var] = driftsArray

 [====================] 100% complete

# Add region mask as a coordinate to the final dataset
This allows the user to easily mask regions when analyzing the data. See the region masking notebook for more information on the utility of this.

In [27]:
#add region mask as coordinate to dataset
is2 = is2.assign_coords(coords = regionMaskCoords)

#add descriptive attributes 
is2.region_mask.attrs = regionMaskAttrs
is2.longitude.attrs = is2LonsAttrs
is2.latitude.attrs = is2LatsAttrs
is2.attrs = {'description':'data used in nicolejkeeney ICESat-2 jupyter book', 'note': 'see individual data variables for references', 'creation date': str(date.today())}

print(is2)

<xarray.Dataset>
Dimensions:                   (time: 12, x: 304, y: 448)
Coordinates:
  * time                      (time) datetime64[ns] 2018-11-01 ... 2020-04-01
    longitude                 (y, x) float32 168.3 168.1 168.0 ... -10.18 -9.999
    latitude                  (y, x) float32 31.1 31.2 31.3 ... 34.58 34.47
    region_mask               (y, x) uint8 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
Dimensions without coordinates: x, y
Data variables: (12/25)
    xgrid                     (time, y, x) float32 -3.838e+06 ... 3.738e+06
    ygrid                     (time, y, x) float32 5.838e+06 ... -5.338e+06
    ice_thickness             (time, y, x) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ice_thickness_unc         (time, y, x) float32 nan nan nan ... nan nan nan
    num_segments              (time, y, x) float32 nan nan nan ... nan nan nan
    mean_day_of_month         (time, y, x) float32 nan nan nan ... nan nan nan
    ...                        ...
    t2m                       (ti

## Save data to local machine 

In [34]:
is2_filename = 'icesat2-book-data.nc'
is2.to_netcdf(path = localDirectory + is2_filename, format = 'NETCDF4', mode = 'w')
print('File ' + '"%s"' % is2_filename + ' saved to directory ' + '"%s"' % localDirectory)

File "icesat2-book-data.nc" saved to directory "/Users/nicolekeeney/github_repos/icesat2-book-data/"
